<a href="https://colab.research.google.com/github/tncourtz/RL-EV-CS-Energy-Management-System/blob/main/Q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Q-Network**

In [ ]:
#import neccessary modules 
import numpy as np
import random
from IPython.display import clear_output
from collections import deque
import progressbar
import gym
from gym import wrappers
import collections
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

# **Q-learning Implementation**

In [ ]:
import numpy as np

gamma = 0.99      #discount factor
alpha = 0.01       # learning rate
e = 1             #e-greedy 

In [ ]:
#intialize Q-value table 
Q = np.array(np.zeros([24, 10]))
R = np.array(np.zeros([24, 10]))

# skip

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving trading_data.xlsx to trading_data.xlsx


In [ ]:
from pandas import read_excel
df = read_excel('trading_data.xlsx')

PVt= df['pv'].values
PLt = df['load'].values
GTt = df['gt'].values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from math import exp

# defines the reward/connection graph
Q = np.array(np.zeros([24, 10]))          # Q value 
R = np.array(np.zeros([24, 10]))          # Reward
C = np.array(np.zeros([24, 10]))          # Total Cost
Cost = []


def action_space(action): #action space
  if action==0:
    soe = 10
  if action==1: 
    soe = 20   
  if action==2:
    soe = 30    
  if action==3:
    soe =40    
  if action==4:
    soe = 50    
  if action==5:
    soe = 60    
  if action==6:
    soe =70    
  if action==7:
    soe =80    
  if action==8:
    soe = 90    
  if action==9:
    soe = 100
  return soe

  
def update_q(state, next_state, action, alpha, gamma, STT, ACT):
    Pbss = STT[4] - ACT # ACT is the SOE
    Pg = (STT[1]-STT[2]) - Pbss
    Cg = STT[3]*Pg
    Cbd = bat_cost(ACT, Pbss, STT[4])
    Cbss = Pbss*Cbd
    C[state, action]  =  Cg + Cbss
    rsa = 1/(Cg +Cbss +1)
    R[state, action] = rsa
    qsa = Q[state, action]
    new_Q = qsa + alpha * ((rsa + gamma * max(Q[next_state, :])) - qsa)
    Q[state, action] = new_Q
    # renormalize row to be between 0 and 1
    rn = Q[state][Q[state] > 0] / np.sum(Q[state][Q[state] > 0])
    Q[state][Q[state] > 0] = rn
    Cost.append(Cg+Cbd)
    return R[state, action], Pg, Pbss

def bat_cost(SOE, Pb,SOE_1):
  #Variables 
  D = 100                               # (kwh) battery full capacity
  Tamb = 23                            #ambient temperature
  Rth = 0.0002                          # battery thermal resistance

  # model parameters (obtained by curve fitting)
  a = 1.1434       
  b = 48.808       
  m = 3.185
  d = 0.0768

  Yhr = 8760                           # hours per year
  Cbat = 200                           # initial battery cost 

  SOCmin = 0.1                         #minimum battery soc
  SOCmax = 0.9                         #maximum battery soc 
  SOCinit = 0.5                        #initial battery soc
  Pbmin = -75                          #minimium battery power discharge
  Pbmax = 75                           #maximum battery power charge
  Pgmin = -100                          #maximum power fed to the grid
  Pgmax = 150                           #minimum power fed to the grid 
  dt = 1                                # 1 hour time intervals

  #SYSTEM DYNAMICS
  T = Tamb + Rth*abs(Pb)                          # battery temperature 
  L = a*exp(b/T)                             # battery lifetime 
  SOCavg = SOE/100                               # average SOC ?what is the avg. soc
  DOD = 1 - SOCavg
  Ctemp = Cbat*dt/(Yhr*L)                         # Temperature operation cost
  Csoc = (Cbat*m*SOCavg-d)/(15*0.2*Yhr)           # Cost due to avgSOC 
  Cdod = (-0.012*DOD**4) + (0.033*DOD**3) + (0.021*DOD**2) - (0.056*DOD) # cost due to dod
  Cbd = max(Ctemp, Cdod, Csoc)                    # total battery degradation cost 
  return Cbd



# Core algorithm
gamma = 0.99
alpha = 0.001
n_episodes = 10E3
n_states = 24
n_actions = 10
epsilon = 1
decay = 0.999
p = 1 - epsilon    # probability used for greedy method 
states = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
act = [0,1,2,3,4,5,6,7,8,9]
PGg =    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
PBb =    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
for e in range(int(n_episodes)):

    # current_state = [0, Plt[0], Ppvt[0], Gt[0], Eb]
    # states.append(current_state)

    k = 0 
    pg = 0
    pb = 0
    while k <= 23:
        if k == 0:
          current_state = [k, PLt[k], PVt[k], GTt[k], 50]
        else:
          current_state = current_state

        states[k] = (current_state)
        PGg[k] = pg
        PBb[k] = pb
        # epsilon greedy
        p = 1-epsilon

        if p < epsilon:
          act = np.random.randint(0, 9)
          action = action_space(act)
        else:
          act = np.argmax(Q[k])
          action = action_space(act)

        # Cbd = 1                   # battery degradation cost
        # action = PLt[k] - PVt[k]
        # act[k] = action
        if k < 23: 
          reward, pg, pb = update_q(k, k+1, act, alpha, gamma, current_state, action)
          next_state = [k+1, PLt[k+1], PVt[k+1], GTt[k+1],action]
          current_state = next_state
        else:
          current_state = current_state
          pg = pg
          pb = pb

        k = k+1

    epsilon = max(0.1, decay*epsilon)

print(states)
print(PGg)
print(PBb)

[[0, 4.67229760398231, 0.15405369172268, 0.0503287611, 50], [1, 0.555466745186294, 0.369655689674999, 0.041195505800000004, 60], [2, 10.0462999245859, -0.0157323452691571, 0.0655712726, 90], [3, 20.432643175396, 0.373265469844085, 0.07665286290000001, 60], [4, 39.746655411696, 0.369686236780211, 0.10528735460000001, 90], [5, 55.6894389606573, 1.09070137335916, 0.029038378400000002, 10], [6, 96.9837742982538, 5.990881138258, 0.08771807620000001, 80], [7, 74.2735287342808, 19.503647525042, 0.1378703991, 20], [8, 40.1656244709645, 34.9660273489275, 0.1364345105, 60], [9, 21.0254549922689, 46.7532947535645, 0.17969784590000001, 50], [10, 57.8629856948504, 59.8239234200317, 0.24198016590000002, 10], [11, 39.6269747829417, 33.093052446633, 0.2065616465, 70], [12, 20.1406975511381, 42.7768835162195, 0.2114097812, 80], [13, 10.1737153976192, 48.9994413563005, 0.1854631847, 50], [14, 20.3666052809042, 43.2687483322401, 0.1938450844, 30], [15, 44.4058525100308, 36.8594382961988, 0.1584966985, 90

In [ ]:
#Grid costs
cost = 0
for i in range(len(PGg)):
  cost += GTt[i]*PGg[i]

print(cost)

68.33557723364662


In [ ]:
#Battery lifetime
from math import exp
Pb = PBb 
L=0
for i in range(24):
  T = 23+ 0.0002*abs(Pb[i])*1000
  L += 1.434*exp(48.808/T)
L = L/24
print(L)

8.277680675837411
